In [ ]:
#The AILEA Code Challenge
#Let's make some stock market analysis here :)

In [1]:
import ftplib
import pandas as pd

def get_symbols():
    #STEP1: this function retrieves teh symbols of interest from the NASDAQ
    #connection to the server
    #username and passwd required!
    ftp = ftplib.FTP('ftp.nasdaqtrader.com', 'Anonymous', 'guest')
    
    #retrieving the data file into our local directory
    ftp.cwd('/symboldirectory/')
    filename = 'nasdaqlisted.txt'
    ftp.retrbinary('RETR ' + filename, open(filename, 'wb').write)
    ftplib.FTP.quit(ftp) #polite exit
    
    #extract just 50 first ones into data frame
    df = pd.read_csv(filename, sep='|', usecols=['Symbol'])
    #df.sort_values('Symbol') #actually useless as the list seems to be in order anyways
    return df.head(50)

companies = get_symbols()
print(companies)

   Symbol
0    AACG
1    AACI
2   AACIU
3   AACIW
4    AADI
5    AADR
6     AAL
7    AAME
8    AAOI
9    AAON
10   AAPB
11   AAPD
12   AAPL
13   AAPU
14   AATC
15   AAWW
16   AAXJ
17   ABCB
18   ABCL
19   ABCM
20   ABEO
21   ABGI
22   ABIO
23   ABMD
24   ABNB
25   ABOS
26   ABSI
27   ABST
28   ABTX
29   ABUS
30   ABVC
31   ACAB
32  ACABU
33  ACABW
34   ACAC
35  ACACU
36  ACACW
37   ACAD
38   ACAH
39  ACAHU
40  ACAHW
41   ACAX
42  ACAXR
43  ACAXU
44  ACAXW
45    ACB
46   ACBA
47  ACBAU
48  ACBAW
49   ACCD
